1. 문서의 내용을 읽는다 (w. Document Loaders)
2. 문서를 쪼갠다
    1. 토큰 수 초과로 답변을 생성하지 못할 수 있고,
    2. 문서가 길면 답변 생성이 오래걸림
3. 임베딩 후 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [21]:
# %pip install python-docx
# %pip install tiktoken

1) 문서 내용 읽기

In [22]:
from docx import Document

document = Document('./tax.docx')
print(f'document: {dir(document)}')

print()

full_text = ''
for idx, paragraph in enumerate(document.paragraphs):
    # print(f'paragraph: {paragraph.text}')
    # if idx == 5: break
    full_text += f'{paragraph.text}\n'

# print(f'full_text: {full_text}')


document: ['_Document__body', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_block_width', '_body', '_element', '_parent', '_part', 'add_heading', 'add_page_break', 'add_paragraph', 'add_picture', 'add_section', 'add_table', 'core_properties', 'element', 'inline_shapes', 'iter_inner_content', 'paragraphs', 'part', 'save', 'sections', 'settings', 'styles', 'tables']



2. 문서 쪼개기 (w. openai의 TicToken)

In [23]:
import tiktoken

encoder = tiktoken.encoding_for_model('gpt-4o')

# encoding: text -> num list
encoding = encoder.encode(full_text)  # 숫자로 된 토큰 리스트 
print(len(encoding))

# decoding: num list -> text
decoded = encoder.decode(encoding)
print(len(decoded))

187982
288073


In [29]:
# 토큰의 length 를 파악하고, 원하는 chunk size 로 쪼개고 다시 문서로 변환
import tiktoken

# spliter 구현
def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model('gpt-4o')
    encoding = encoder.encode(full_text)
    total_token_count = len(encoding)

    text_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = encoding[i:i+chunk_size]  # chunk size 만큼 쪼개기 
        decoded = encoder.decode(chunk)
        text_list.append(decoded)

    return text_list

chunk_list = split_text(full_text, 1500)


In [30]:
print(len(chunk_list))
print(chunk_list[0])

126


소득세법
[시행 2024. 7. 1.] [법률 제19933호, 2023. 12. 31., 일부개정]

기획재정부(재산세제과(양도소득세)) 044-215-4312
기획재정부(소득세제과(근로소득)) 044-215-4216
기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233
기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217

       제1장 총칙 <개정 2009. 12. 31.>
제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.
  [본조신설 2009. 12. 31.]
  [종전 제1조는 제2조로 이동 <2009. 12. 31.>]
제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>
  1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.
  2. “비거주자”란 거주자가 아닌 개인을 말한다.
  3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.
  4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.
  5. “사업자”란 사업소득이 있는 거주자를 말한다.
  ② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.
  [본조신설 2009. 12. 31.]
제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.
  1. 거주자
  2. 비거주자로서 국내원천소득(國內源泉所得)이 있는 개인
  ② 다음 각 호의 어느 하나에 해당하는 자는 이 법에 따라 원천징수한 소득세를 납부할 의무를 진다.
  1. 거주자
  2. 비거주자
  3. 내국법인
  4. 외국법인의 국내지점 또는 국내영업소(출장소, 그 밖에 이에 준하는 

3. 문서 임베딩 

In [32]:
import chromadb

chroma_client = chromadb.Client()

collection_name = 'tax_collection'
# tax_collection = chroma_client.create_collection(collection_name) # collection: 테이블 비슷한 개념

In [33]:

import os
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(
    api_key = openai_api_key, 
    model_name = 'text-embedding-3-large'
    )

tax_collection = chroma_client.get_or_create_collection(
    collection_name, 
    embedding_function = openai_embedding
    )

id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')

tax_collection.add(documents = chunk_list, ids=id_list)

4. 유사도 검색

In [35]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_doc = tax_collection.query(query_texts=query, n_results=3)

print(retrieved_doc['documents'][0])

[' 배당소득에 대해서는 100분의 14\n  3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n  4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n  5. 공적연금소득에 대해서는 기본세율\n  5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n    가. 연금소득자의 나이에 따른 다음의 세율\n      \n    나. 삭제 <2014. 12. 23.>\n    다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n  5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n    가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n    나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n  6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n    가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n    나. 제21조제1항제18호 및 제21호에 따른 기타소득에 대해서는 100분의 15\n    다. 삭제 <2014. 12. 23.>\n    라. 그 밖의 기타소득에 대해서는 100분의 20\n  7.

5. LLM 질의

In [37]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o", # 위에 문서 split 할 때 썼던 모델 
  messages=[
    {"role": "system", "content": f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요 {retrieved_doc['documents'][0]}"},
    {"role": "user", "content": query}
  ]
)

print(response.choices[0].message.content)

연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 기본세율을 적용해야 합니다. 한국의 소득세는 누진세율 구조로, 연간 소득 구간에 따라 세율이 다릅니다. 2023년 기준으로 소득세 과세표준과 세율은 다음과 같습니다:

- 1,200만원 이하: 6%
- 1,200만원 초과 4,600만원 이하: 15%
- 4,600만원 초과 8,800만원 이하: 24%
- 8,800만원 초과 1억5천만원 이하: 35%
- 1억5천만원 초과: 38%

연봉 5천만원에 대한 소득세 계산은 다음 단계로 이루어집니다:

1. 1,200만원 이하 부분: 1,200만원 * 6% = 72만원
2. 1,200만원 초과 4,600만원 이하 부분: (4,600만원 - 1,200만원) * 15% = 510만원
3. 4,600만원 초과 5,000만원 이하 부분: (5,000만원 - 4,600만원) * 24% = 96만원

각 부분의 세금을 더하면:

72만원 + 510만원 + 96만원 = 678만원

이를 통해 연봉 5천만원일 경우, 기본 소득세 계산이 678만원이 됩니다. 다만 실제로 납부해야 할 소득세는 건강보험료, 국민연금 등의 공제와 세액공제, 각종 감면 등을 고려해야 하므로, 최종 세액은 이보다 줄어들 수 있습니다. 정확한 세액을 알기 위해서는 연말정산을 통해 계산되는 것이 필요합니다.
